In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset

In [2]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\notebooks\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [3]:
df = pd.read_excel("references\Subject_info_balanced.xlsx")

In [4]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [6]:
class VGG3D(nn.Module):
	def __init__(self, num_classes=2, input_shape=(1,110,110,110)): # input: input_shape:	[num_of_filters, kernel_size] (e.g. [256, 25])
		super(VGG3D, self).__init__()
		self.conv1 = nn.Sequential(
			nn.Conv3d(
				in_channels=input_shape[0],        # input height
				out_channels=8,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=8,        # input height
				out_channels=8,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1              
            ),
			nn.ReLU(),                  # activation
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)
        
		self.conv2 = nn.Sequential(
			nn.Conv3d(
				in_channels=8,        # input height
				out_channels=16,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1              
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=16,        # input height
				out_channels=16,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
			nn.ReLU(),                  # activation
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)
        
		self.conv3 = nn.Sequential(
			nn.Conv3d(
				in_channels=16,        # input height
				out_channels=32,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1               
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=32,        # input height
				out_channels=32,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
			nn.ReLU(),                  # activation
            nn.Conv3d(
				in_channels=32,        # input height
				out_channels=32,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1              
			),
			nn.ReLU(),
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)

		self.conv4 = nn.Sequential(
			nn.Conv3d(
				in_channels=32,        # input height
				out_channels=64,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
            nn.ReLU(),
            nn.Conv3d(
				in_channels=64,        # input height
				out_channels=64,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1                
			),
			nn.ReLU(),                  # activation
            nn.Conv3d(
				in_channels=64,        # input height
				out_channels=64,       # n_filters
				kernel_size=(3,3,3),          # filter size
				padding=1
			),
			nn.ReLU(),
			nn.MaxPool3d(kernel_size=(2,2,2), stride=2) # choose max value in 2x2 area
		)
		
		fc1_output_features=128     
		self.fc1 = nn.Sequential(
			 nn.Linear(13824, 128),
 			 nn.BatchNorm1d(128),            
			 nn.ReLU()
		 )

		fc2_output_features=64           
		self.fc2 = nn.Sequential(
			 nn.Linear(fc1_output_features, fc2_output_features),
			 nn.BatchNorm1d(fc2_output_features),
			 nn.ReLU()
		 )

		if(num_classes==2):
			self.out = nn.Linear(fc2_output_features, 2)
			self.out_act = nn.Sigmoid()
		else:
			self.out = nn.Linear(fc2_output_features, num_classes)
			self.out_act = nn.Softmax()


	def forward(self, x, drop_prob=0.8):

		x = self.conv1(x)
#		print(x.shape)        
		x = self.conv2(x)
#		print(x.shape)        
		x = self.conv3(x)
#		print(x.shape)        
		x = self.conv4(x)
#		print(x.shape)        
		x = x.view(x.size(0), -1)           # flatten the output of conv2 to (batch_size, num_filter * w * h)
#		print(x.shape)        
		x = self.fc1(x)
		x = nn.Dropout(drop_prob)(x)
		x = self.fc2(x)
		#x = nn.Dropout(drop_prob)(x)        
		prob = self.out(x) # probability
# 		y_hat = self.out_act(prob) # label
# 		return y_hat, prob, x    # return x for visualization
		return prob

In [7]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim


def run_experiment(df, config):
    """Run the experiment with the given configuration on the preprocessed DataFrame."""
    train_dataset, val_dataset, test_dataset = load_datasets(df, config['image_type'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    # Initialize model and training components
    device = torch.device("cuda")
    model = VGG3D().to(device)
    
    criterion = torch.nn.CrossEntropyLoss()

    #criterion = nn.BCEWithLogitsLoss()
    #optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    optimizer = torch.optim.Adam(net.parameters(), lr=config['lr'], weight_decay=0.0)
#       optimizer = torch.optim.SGD(net.parameters(), lr=LR, momentum=0.9)

    # Training and validation
    train_accuracies, val_accuracies, val_losses = train_and_validate(model, train_loader, val_loader, criterion, optimizer, label_mapping, config['num_epochs'], config['patience'], device)
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device)
    
    
    
    
  # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')

    # Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            # Reindex the existing DataFrame to ensure all columns are aligned and add new columns if necessary
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    return filename, train_accuracies[-1], val_accuracies[-1], test_accuracy

# Example configuration and use case
config = {
    'optimizer': 'Adam',
    'lr' : 0.01,
    'loss_criterion': 'Cross-Entropy',
    'num_epochs': 10,
    'batch_size': 1,
    'patience': 4,
    'Description' : 'VG33D advanced',

}

In [8]:
image_types = ['MRI_PET', 'MRI', 'PET']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)


Working on image type: MRI_PET


Epoch 1/10 - Train:   0%|          | 0/76 [00:00<?, ?it/s]c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\nn\modules\conv.py:605: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv3d(
Epoch 1/10 - Train:   0%|          | 0/76 [00:05<?, ?it/s]


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])